<a href="https://colab.research.google.com/github/Arcaneless/GAR_Efficient_Net_B0/blob/main/Labeling_People_Counting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro




# Configure GPU

In [ ]:
# CUDA: Let's check that Nvidia CUDA drivers are already pre-installed and which version is it.
!/usr/local/cuda/bin/nvcc --version
# We need to install the correct cuDNN according to this output

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [ ]:
#take a look at the kind of GPU we have
!nvidia-smi

Thu Oct 22 07:59:24 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Change the number depending on what GPU is listed above, under NVIDIA-SMI > Name.
# Tesla K80: 30
# Tesla P100: 60
# Tesla T4: 75
%env compute_capability=61

env: compute_capability=61


# Install DarkNet

In [ ]:
%cd /content/
%rm -rf darknet

/content


In [ ]:
#we clone the fork of darknet maintained by roboflow
#small changes have been made to configure darknet for training
!git clone https://github.com/roboflow-ai/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 13289, done.
remote: Total 13289 (delta 0), reused 0 (delta 0), pack-reused 13289
Receiving objects: 100% (13289/13289), 12.13 MiB | 24.78 MiB/s, done.
Resolving deltas: 100% (9107/9107), done.


In [ ]:
#install environment from the Makefile
%cd darknet/
# compute_30, sm_30 for Tesla K80
# compute_75, sm_75 for Tesla T4
#!sed -i 's/ARCH= -gencode arch=compute_60,code=sm_60/ARCH= -gencode arch=compute_30,code=sm_30/g' Makefile

#install environment from the Makefile
#note if you are on Colab Pro this works on a P100 GPU
#if you are on Colab free, you may need to change the Makefile for the K80 GPU
#this goes for any GPU, you need to change the Makefile to inform darknet which GPU you are running on.
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
!sed -i "s/ARCH= -gencode arch=compute_61,code=sm_61/ARCH= -gencode arch=compute_${compute_capability},code=sm_${compute_capability}/g" Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:910:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1391:14: warning: unused variable ‘buff’ [-Wunused-variable]
         char buff[100];
              ^~~~
./src/image_opencv.cpp:1367:9: warning: unused variabl

# Download and extract frames from Youtube

In [ ]:
!pip install youtube_dl

import youtube_dl

     |████████████████████████████████| 1.8MB 8.2MB/s 


In [ ]:
# checking video formats
!youtube-dl -F https://www.youtube.com/watch?v=yI7lYLe2I0c

[youtube] yI7lYLe2I0c: Downloading webpage
[youtube] yI7lYLe2I0c: Downloading MPD manifest
[info] Available formats for yI7lYLe2I0c:
format code  extension  resolution note
251          webm       audio only DASH audio    9k , webm_dash container, opus @160k (48000Hz)
139          m4a        audio only DASH audio   65k , m4a_dash container, mp4a.40.5@ 48k (22050Hz)
140          m4a        audio only DASH audio  144k , m4a_dash container, mp4a.40.2@128k (44100Hz)
278          webm       256x144    DASH video   95k , webm_dash container, vp9, 25fps, video only
160          mp4        256x144    DASH video  108k , mp4_dash container, avc1.4d400b, 25fps, video only
242          webm       426x240    DASH video  220k , webm_dash container, vp9, 25fps, video only
133          mp4        426x240    DASH video  242k , mp4_dash container, avc1.4d400c, 25fps, video only
243          webm       640x360    DASH video  405k , webm_dash container, vp9, 25fps, video only
134          mp4        640x3

In [ ]:
# setting all the source from youtube
heading = 'https://www.youtube.com/watch?v='
fps = 25
youtube_data = [
                {
                    'link': 'https://www.youtube.com/watch?v=yI7lYLe2I0c',
                    'id': 'yI7lYLe2I0c',
                    'format': '135',
                    'start-second': 0,
                    'end-second': 4347
                },
                {
                    'link': 'https://www.youtube.com/watch?v=Zh3MKmK6Jmo',
                    'id': 'Zh3MKmK6Jmo',
                    'format': '135',
                    'start-second': 6,
                    'end-second': 60
                },
                {
                    'link': 'https://www.youtube.com/watch?v=MSmtkpvKmnI',
                    'id': 'MSmtkpvKmnI',
                    'format': '135',
                    'start-second': 0,
                    'end-second': 40
                },
                {
                    'link': 'https://www.youtube.com/watch?v=Jdu-cfxZXBA',
                    'id': 'Jdu-cfxZXBA',
                    'format': '135',
                    'start-second': 6,
                    'end-second': 70
                },
                {
                    'link': 'https://www.youtube.com/watch?v=-a2bIhr-sNc',
                    'id': '-a2bIhr-sNc',
                    'format': '135',
                    'start-second': 37,
                    'end-second': 72
                }
]

In [ ]:
%cd /content/youtube_data
import os

# this is the download code cell of all of the youtube video

for data in youtube_data:
  id = data['id']
  path = f'./{id}.mp4'
  if (os.path.isfile(path)): continue
  ydl_opts = {
      'format': data['format'],
      'outtmpl': path
  }
  with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download([data['link']])

[Errno 2] No such file or directory: '/content/youtube_data'
/content/darknet
[youtube] yI7lYLe2I0c: Downloading webpage
[youtube] yI7lYLe2I0c: Downloading MPD manifest
[dashsegments] Total fragments: 851
[download] Destination: ./yI7lYLe2I0c.mp4
[download]  37.8% of ~604.72MiB at 43.89MiB/s ETA 08:12

ERROR: giving up after 0 retries


[download] Skipping fragment 323...
[download]  70.9% of ~606.83MiB at 44.26MiB/s ETA 03:56[download] Skipping fragment 605...
[download] Skipping fragment 606...
[download]  76.0% of ~606.75MiB at 37.23MiB/s ETA 03:16

ERROR: giving up after 0 retries


[download] Skipping fragment 651...
[download] Skipping fragment 652...
[download]  76.6% of ~606.63MiB at 48.80MiB/s ETA 03:11[download] Skipping fragment 658...
[download]  77.0% of ~606.72MiB at 56.66MiB/s ETA 03:08[download] Skipping fragment 662...
[download]  81.9% of ~606.65MiB at 49.79MiB/s ETA 02:28[download] Skipping fragment 705...
[download]  83.7% of ~606.69MiB at 45.25MiB/s ETA 02:14[download] Skipping fragment 721...
[download]  91.3% of ~606.97MiB at 57.76MiB/s ETA 01:12[download] Skipping fragment 787...
[download]  92.8% of ~607.14MiB at 52.21MiB/s ETA 00:59[download] Skipping fragment 801...
[download]  94.9% of ~606.95MiB at 35.98MiB/s ETA 00:42[download] Skipping fragment 820...
[download] 100% of 597.88MiB in 13:38
[youtube] Zh3MKmK6Jmo: Downloading webpage
[youtube] Zh3MKmK6Jmo: Downloading MPD manifest
[dashsegments] Total fragments: 14
[download] Destination: ./Zh3MKmK6Jmo.mp4
[download] 100% of 3.23MiB in 00:12
[youtube] MSmtkpvKmnI: Downloading webpage
[youtu

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import random

# this function extracts all youtube video with random frames given
def number_of_frames_needed(index, frames_num):
  data = youtube_data[index]
  frames = list(range(data['start-second']*fps, data['end-second']*fps))
  random.shuffle(frames)

  id = data['id']
  cap = cv2.VideoCapture(f"/content/darknet/{id}.mp4")
  # get total number of frames
  totalFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
  data['total_frames'] = totalFrames
  for i in range(frames_num):
    frame_to_set = frames[i]
    print(f'Reading the {id}\'s {frame_to_set} frame')

    # check for valid frame number
    if frame_to_set >= 0 & frame_to_set <= totalFrames:
      # set frame position
      cap.set(cv2.CAP_PROP_POS_FRAMES,frame_to_set)
    else:
      raise ValueError('Frame is out of bound')

    ret, frame = cap.read()

    dir = f'/content/dataset1/{id}'
    if (not os.path.isdir(dir)): os.mkdir(dir)
    cv2.imwrite(f'{dir}/{str(frame_to_set)}.jpg',frame)
  cap.release()
      

In [ ]:
%rm -rf /content/dataset1
# save all image to dataset1 folder
try:
  if (not os.path.isdir('/content/dataset1')): os.mkdir('/content/dataset1')
  for i in range(len(youtube_data)):
    number_of_frames_needed(i, 200)
except ValueError:
  print('Error occured')

Reading the yI7lYLe2I0c's 105086 frame
Reading the yI7lYLe2I0c's 96270 frame
Reading the yI7lYLe2I0c's 78648 frame
Reading the yI7lYLe2I0c's 73784 frame
Reading the yI7lYLe2I0c's 64308 frame
Reading the yI7lYLe2I0c's 28577 frame
Reading the yI7lYLe2I0c's 57392 frame
Reading the yI7lYLe2I0c's 91546 frame
Reading the yI7lYLe2I0c's 56206 frame
Reading the yI7lYLe2I0c's 36434 frame
Reading the yI7lYLe2I0c's 94169 frame
Reading the yI7lYLe2I0c's 19736 frame
Reading the yI7lYLe2I0c's 44459 frame
Reading the yI7lYLe2I0c's 75981 frame
Reading the yI7lYLe2I0c's 18059 frame
Reading the yI7lYLe2I0c's 58680 frame
Reading the yI7lYLe2I0c's 3396 frame
Reading the yI7lYLe2I0c's 4665 frame
Reading the yI7lYLe2I0c's 85840 frame
Reading the yI7lYLe2I0c's 15311 frame
Reading the yI7lYLe2I0c's 102252 frame
Reading the yI7lYLe2I0c's 6380 frame
Reading the yI7lYLe2I0c's 95546 frame
Reading the yI7lYLe2I0c's 13504 frame
Reading the yI7lYLe2I0c's 40930 frame
Reading the yI7lYLe2I0c's 19021 frame
Reading the y

# Loading Datas

In [ ]:
%cd /content/darknet
!rm -rf train/ valid/ test/

!wget wget https://pjreddie.com/media/files/yolov3-tiny.weights

/content/darknet
--2020-10-19 07:52:32--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2020-10-19 07:52:32--  https://pjreddie.com/media/files/yolov3-tiny.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35434956 (34M) [application/octet-stream]
Saving to: ‘yolov3-tiny.weights’

yolov3-tiny.weights 100%[===================>]  33.79M   163KB/s    in 4m 52s  

2020-10-19 07:57:26 (119 KB/s) - ‘yolov3-tiny.weights’ saved [35434956/35434956]

FINISHED --2020-10-19 07:57:26--
Total wall clock time: 4m 54s
Downloaded: 1 files, 34M in 4m 52s (119 KB/s)


In [ ]:
%cd /content/darknet
!wget https://pjreddie.com/media/files/yolov3.weights

/content/darknet
--2020-10-19 07:58:56--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  72.1KB/s    in 45m 38s 

2020-10-19 08:44:36 (88.5 KB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [ ]:
!./darknet detect cfg/yolov3.cfg '/content/drive/My Drive/Colab Notebooks/yolov3.weights' /content/dataset1/Jdu-cfxZXBA/230.jpg

In [ ]:
!cp '/content/drive/My Drive/Colab Notebooks/yolov3.weights' '/content/darknet'

In [ ]:
!pip install  opencv-python --upgrade

Requirement already up-to-date: opencv-python in /usr/local/lib/python3.6/dist-packages (4.4.0.44)


In [ ]:
import numpy as np
import cv2
import time
from google.colab.patches import cv2_imshow

# load the COCO class labels our YOLO model was trained on
labelsPath = ("/content/darknet/cfg/coco.names")
LABELS = open(labelsPath).read().strip().split("\n")
#print(LABELS)

# initialize a list of colors to represent each possible class label
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
	dtype="uint8")

# derive the paths to the YOLO weights and model configuration
weightsPath = ("/content/darknet/yolov3.weights")
configPath = ( "/content/darknet/cfg/yolov3.cfg")

# load our YOLO object detector trained on COCO dataset (80 classes)
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)



def load_label_image(path):
  # load our input image and grab its spatial dimensions
  image = cv2.imread(path)
  (H, W) = image.shape[:2]
  print("The image shape is:",image.shape)
  # determine only the *output* layer names that we need from YOLO
  ln = net.getLayerNames()
  # print("ln at start::",ln)
  ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
  # print("ln at end:",ln)

  # construct a blob from the input image and then perform a forward
  # pass of the YOLO object detector, giving us our bounding boxes and
  # associated probabilities
  blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),
	  swapRB=True, crop=False)
  net.setInput(blob)
  start = time.time()
  layerOutputs = net.forward(ln)
  end = time.time()
  #print("The layers are:",layerOutputs)

  # show timing information on YOLO
  print("[INFO] YOLO took {:.6f} seconds".format(end - start))

  # initialize our lists of detected bounding boxes, confidences, and
  # class IDs, respectively
  bounding_box = []
# filtering
  # loop over each of the layer outputs
  for output in layerOutputs:
	  # loop over each of the detections
	  for detection in output:
		  #print(detection)
		  # extract the class ID and confidence (i.e., probability) of
		  # the current object detection
		
		  scores = detection[5:]
		  classID = np.argmax(scores)
		  confidence = scores[classID]
		  if confidence!=0:
			  print(confidence)


		  # filter out weak predictions by ensuring the detected
		  # probability is greater than the minimum probability
		   if confidence > 0.40 and classID == 0:
            # if it matchs coco[0] which is Human
            # scale the bounding box coordinates back relative to the
		    # size of the image, keeping in mind that YOLO actually
			  # returns the center (x, y)-coordinates of the bounding
			  # box followed by the boxes' width and height
            box = detection[0:4] * np.array([W, H, W, H])

			  (centerX, centerY, width, height) = box.astype("int")

			  # use the center (x, y)-coordinates to derive the top and
			  # and left corner of the bounding box
			  x = int(centerX - (width / 2)) # left top x,y
			  y = int(centerY - (height / 2))

			  # update our list of bounding box coordinates
        bounding_box = bounding_box.append([0] + detection[0:4])

		

  return bounding_box



    



# # apply non-maxima suppression to suppress weak, overlapping bounding
# # boxes

# idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.4,
# 	0.3)

# # ensure at least one detection exists
# if len(idxs) > 0:
# 	# loop over the indexes we are keeping
# 	for i in idxs.flatten():
# 		# extract the bounding box coordinates
# 		(x, y) = (boxes[i][0], boxes[i][1])
# 		(w, h) = (boxes[i][2], boxes[i][3])

# 		# draw a bounding box rectangle and label on the image
# 		color = [int(c) for c in COLORS[classIDs[i]]]
# 		cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
# 		text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
# 		cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
# 			0.5, color, 2)

# # show the output image
# #cv2.imwrite("images_result.jpg",image) 

# cv2_imshow(image)



IndentationError: ignored

In [ ]:
driveBasePath = '/content/drive/My Drive/ImagesOutput'

# just move all image over, do not run normally
from distutils.dir_util import copy_tree
import os

for data in youtube_data:
  id = data['id']
  
  # copy subdirectory example
  fromDirectory = f"/content/dataset1/{id}"
  toDirectory = f'{driveBasePath}/{id}'

  os.mkdir(toDirectory)

  copy_tree(fromDirectory, toDirectory)



In [ ]:
# Main Run Cell

def save_YOLO_txt_to_drive(path, bboxes):
  target = f'{path}/'
  with open(path, 'w') as f:
      for row in bboxes:
        f.write(' '.join([str(x) for x in row])+'\n')




for data in youtube_data:
  # get list of jpgs
  id = data['id']
  path = f'/content/dataset1/{id}'
  listOfJPG = os.listdir(path)

  # get dir of txt
  txtPath = f'/content/drive/My Drive/ImagesOutput/{id}/txt'
  if(not os.path.isdir(txtPath)): os.mkdir(txtPath)

  for name in listOfJPG:
    jpg = f'{path}/{name}'
    bounding_boxes = load_label_image(jpg)
    to_txt = txtPath+'/'+name.replace('jpg', 'txt')
    save_YOLO_txt_to_drive(to_txt, bounding_boxes)
    print(f'Saved {to_txt}')

The image shape is: (480, 854, 3)
[INFO] YOLO took 1.788338 seconds
Saved /content/drive/My Drive/ImagesOutput/yI7lYLe2I0c/txt/41020.txt
The image shape is: (480, 854, 3)
[INFO] YOLO took 1.654759 seconds
Saved /content/drive/My Drive/ImagesOutput/yI7lYLe2I0c/txt/96270.txt
The image shape is: (480, 854, 3)
[INFO] YOLO took 1.688702 seconds
Saved /content/drive/My Drive/ImagesOutput/yI7lYLe2I0c/txt/60027.txt
The image shape is: (480, 854, 3)
[INFO] YOLO took 1.723651 seconds
0.22445728
Saved /content/drive/My Drive/ImagesOutput/yI7lYLe2I0c/txt/15311.txt
The image shape is: (480, 854, 3)
[INFO] YOLO took 1.688654 seconds
Saved /content/drive/My Drive/ImagesOutput/yI7lYLe2I0c/txt/57677.txt
The image shape is: (480, 854, 3)
[INFO] YOLO took 1.689659 seconds
Saved /content/drive/My Drive/ImagesOutput/yI7lYLe2I0c/txt/50261.txt
The image shape is: (480, 854, 3)
[INFO] YOLO took 1.666028 seconds
Saved /content/drive/My Drive/ImagesOutput/yI7lYLe2I0c/txt/105086.txt
The image shape is: (480, 854